In [1]:
from datetime import datetime
import time

import gdax
import numpy as np
import pandas as pd

In [2]:
order_book = gdax.OrderBook(product_id='BTC-USD')

In [3]:
def get_volume(orders):
    return np.sum([np.float(order['size']) for order in orders])

def get_ask_volume(order_book):
    return get_volume(order_book.get_asks(order_book.get_ask()))

def get_bid_volume(order_book):
    return get_volume(order_book.get_bids(order_book.get_bid()))

In [4]:
bids = []
asks = []
bid_volumes = []
ask_volumes = []
times = []

In [7]:
order_book.start()
print("Doing initial sleep...")
time.sleep(10)
print("Collecting data...")
try:
    while True:
        bids += [np.float(order_book.get_bid())]
        asks += [np.float(order_book.get_ask())]
        bid_volumes += [get_bid_volume(order_book)]
        ask_volumes += [get_ask_volume(order_book)]
        times += [datetime.now()]
        if len(times) % 20 == 0:
            print(len(times))
        time.sleep(0.5)
except KeyboardInterrupt as e:
    order_book.close()
    raise(e)

-- Subscribed to OrderBook! --

Doing initial sleep...
20
40
60
80
100

-- OrderBook Socket Closed! --


KeyboardInterrupt: 

In [ ]:
datetime.now()

In [10]:
data = pd.DataFrame({'bid': bids,
                     'ask': asks,
                     'bid-volume': bid_volumes,
                     'ask-volume': ask_volumes,
                     'time': times})
data

,bid,ask,bid-volume,ask-volume,time
0,6478.93,6478.94,5.619360,2.725390,2018-06-13 22:50:43.287370
1,6478.93,6478.94,12.101788,3.179960,2018-06-13 22:51:22.773969
2,6478.93,6478.94,6.157315,3.911780,2018-06-13 22:52:33.142885
3,6478.93,6478.94,6.157315,4.911780,2018-06-13 22:52:33.643658
4,6478.93,6478.94,6.157315,4.911780,2018-06-13 22:52:34.145539
5,6478.93,6478.94,6.157315,4.911780,2018-06-13 22:52:34.650381
6,6478.93,6478.94,6.157315,4.911780,2018-06-13 22:52:35.152188
7,6478.93,6478.94,6.157315,4.911780,2018-06-13 22:52:35.653636
8,6478.93,6478.94,6.157315,4.813880,2018-06-13 22:52:36.154866
9,6478.93,6478.94,6.157315,4.813880,2018-06-13 22:52:36.656302


In [11]:
data.to_csv("data/test.csv")